In [ ]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0, "../")
sys.path.insert(0,"../pompjax/pompjax/")

from global_config import config

results_dir           = config.get_property('results_dir')
results2_dir          = config.get_property('results2_dir')

data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")

COLOR_LIST1           = ["#F8AFA8", "#FDDDA0", "#F5CDB4", "#74A089"]

from utils_local.misc import amro2title, amro2cute

In [ ]:
def empirical_prevalence(amro, path_to_prev="../data/amro_prevalence.csv"):
    amro_prev_df = pd.read_csv(path_to_prev)
    gamma        = amro_prev_df[amro_prev_df.amro==amro]["prevalence_mean1"].values[0]/100
    return gamma

def simulate_abm(f, f0, g, θ, model_settings):
    dates_simulation = model_settings["dates_simulation"]

    x = f0(θ)

    observations          = np.full((len(dates_simulation), model_settings["k"], model_settings["m"]), np.nan)
    observations[0, :, :] = g(0, x, θ)

    for t, date in enumerate(dates_simulation[1:]):
        x                       = f(t, x, θ)
        observations[t+1, :, :] = g(t, x, θ)
    return observations

def create_obs_infer(obs_sim, idx_infer, dates, model_settings, resample="W-Sun"):
    # obs_sim \in R^{[k x T x m]} as required by pompjax
    infer_df = pd.DataFrame(index=dates)
    for i in range(model_settings["k"]) :
        infer_df['y'+str(i+1)]   = obs_sim[i, :, idx_infer]
        infer_df['oev'+str(i+1)] = 1 +(0.2 * infer_df['y'+str(i+1)].values)**2
    infer_df                     = infer_df.resample(resample).sum()
    infer_df.index.values[-1]    = model_settings["dates"][-1]
    return infer_df

In [ ]:
dates_simulation = pd.date_range(start="2020-02-01", end="2021-02-28", freq="D")

movement_df                  = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", 'patient_movement_2022-Nov.csv'), parse_dates=['date']).drop_duplicates(subset=["date", "mrn"], keep="first")
movement_df["ward_total"]    = movement_df.apply(lambda x: x["ward"]+"-"+x["building"]+"-"+x["place"], axis=1)
movement_df                  = movement_df[movement_df["date"].isin(dates_simulation)]

mrd2id                       = {mrn: id for id, mrn in enumerate(movement_df.mrn.unique())}
ward2id                      = {ward_name: id for id, ward_name in enumerate(np.sort(movement_df.ward_total.unique()))}

movement_df["mrn_id"]        = movement_df.mrn.map(mrd2id)
movement_df["ward_id"]       = movement_df.ward_total.map(ward2id)

ward_size_df                 = movement_df.reset_index()
ward_size_df["ward_id"]      = ward_size_df["ward_total"].apply(lambda x: ward2id[x])
ward_size_df["num_patients"] = 1
ward_size_df                 = ward_size_df.groupby(["date", "ward", "ward_id"]).sum()[["num_patients"]].reset_index().drop(columns=["date"])
ward_size_df                 = ward_size_df.groupby(["ward", "ward_id"]).mean().reset_index().sort_values(by="num_patients")
ward2size                    = {r.ward_id: r.num_patients for idx_r, r in ward_size_df.iterrows()}

id2ward                      = dict((v, k) for k, v in ward2id.items())

###-###-###-###-###-###-###-###-###-###-###-###

selected_buildings = ['Allen Hospital-Allen', 'Harkness Pavilion-Columbia', 'Milstein Hospital-Columbia', 'Mschony-Chony', 'Presbyterian Hospital-Columbia']
building2id        = {selected_buildings[i]: i for i in range(len(selected_buildings))}

def building2observation(building):
    if building in selected_buildings:
        return building2id[building]
    else:
        return 5

ward_names                   = np.sort(list(movement_df.ward_total.unique()))
ward_names_df                = pd.DataFrame(ward_names, columns=["ward"])
ward_names_df                = pd.DataFrame(ward_names, columns=["ward"])
ward_names_df["building"]    = ward_names_df["ward"].apply(lambda x: "-".join(x.split("-")[1:]))
ward_names_df["buidling_id"] = ward_names_df["building"].apply(lambda x: building2observation(x) )
ward_names_df["ward_id"]     = ward_names_df.apply(lambda x: np.where(ward_names_df.ward == x.ward)[0][0], axis=1)

###-###-###-###-###-###-###-###-###-###-###-###

selected_buildings     = ['Allen Hospital-Allen', 'Harkness Pavilion-Columbia', 'Milstein Hospital-Columbia', 'Mschony-Chony', 'Presbyterian Hospital-Columbia']
building2id            = {selected_buildings[i]: i for i in range(len(selected_buildings))}
wardid2buildingid      = {row.ward_id: row.buidling_id for i, row in ward_names_df.iterrows()}
ward2buildingid        =  {row.ward: row.buidling_id for i, row in ward_names_df.iterrows()}
movement_df["cluster"] = movement_df.ward_id.map(wardid2buildingid)


In [ ]:
from diagnostic_plots import convergence_plot
from utils import create_df_response
from ifeakf import ifeakf

def run_amro_inference(f, f0, g, obs_df, model_settings, if_settings, id_run=0, path_to_save=None):
    dates        = pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="D")

    ρmin              = 0.01/2 # test sensitivity minimum
    ρmax              = 0.2    # test sensitivity maximum
    βmin              = 0.00   # transmission rate minimum
    βmax              = 0.11   # transmission rate maximum

    state_space_range = np.array([0, 1])
    parameters_range  = np.array([[ρmin, ρmax],    [βmin, βmax]])
    σ_perturb         = np.array([(ρmax - ρmin)/4, (βmax - βmin)/4]) # (i hve the gut feeling that 0.25 is too large)

    θmle, θpost = ifeakf(process_model                = f,
                            state_space_initial_guess = f0,
                            observational_model       = g,
                            observations_df           = obs_df,
                            parameters_range          = parameters_range,
                            state_space_range         = state_space_range,
                            model_settings            = model_settings,
                            if_settings               = if_settings,
                            perturbation              = σ_perturb)

    np.savez_compressed(os.path.join(path_to_save, f"{str(id_run).zfill(3)}posterior.npz"),
                                    mle           = θmle,
                                    posterior     = θpost)

    ρ_df = create_df_response(θpost[0, :, :, :].mean(-2).T, time=if_settings["Nif"])
    β_df = create_df_response(θpost[1, :, :, :].mean(-2).T, time=if_settings["Nif"])

    p_dfs             = [ρ_df, β_df]
    param_label       = ["ρ", "β"]
    parameters_range  = np.array([[ρmin, ρmax], [βmin, βmax]])

    convergence_plot(θmle, p_dfs, parameters_range, param_label, param_truth=None,
                        path_to_save=os.path.join(path_to_save, f"{str(id_run).zfill(3)}convergence.png"))


In [ ]:
from models import amr_abm, observe_cluster_individual, observe_cluster_population

if_settings = {
        "Nif"                : 20,          # number of iterations of the IF
        "type_cooling"       : "geometric", # type of cooling schedule
        "shrinkage_factor"   : 0.9,         # shrinkage factor for the cooling schedule
        "inflation"          : 1.01         # inflation factor for spreading the variance after the EAKF step
        }

dates_simulation = pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="D")

model_settings   = {
                    "m"                 : 200,
                    "p"                 : 2,
                    "n"                 : movement_df.mrn_id.unique().shape[0],
                    "k"                 : movement_df.cluster.unique().shape[0],
                    "dates"             : pd.date_range(start="2020-02-01", end="2021-02-28", freq="D"),
                    "dates_simulation"  : pd.date_range(start="2020-02-01", end="2021-02-28", freq="D"),
                    "T"                 : len(dates_simulation),  # time to run
                    "num_build"         : len(np.unique(list(wardid2buildingid.values()))),
                    "k"                 : len(np.unique(list(wardid2buildingid.values())))# observing at the building aggregation
                }

assim_dates                       = list(pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="W-Sun"))
assim_dates[-1]                   = dates_simulation[-1]
if_settings["assimilation_dates"] = assim_dates
id_run                            = 0


In [ ]:
def create_obs_building_amro(amro, model_settings, path_to_data, resample="W-Sun", p_oev=0.2):
    k                     = model_settings["k"]
    amro_df               = pd.read_csv(path_to_data, parse_dates=["date"]).drop(columns=["Unnamed: 0"])
    amro_df               = amro_df[amro_df["amro"]==amro]

    amro_df["buildingid"] = amro_df["ward_total"].map(ward2buildingid)

    amro_df               = amro_df.groupby(["date", "buildingid"]).sum(numeric_only=True).unstack([1]).resample(resample).sum(numeric_only=True).stack().reset_index()
    amro_df["obs_name"]   = amro_df["buildingid"].map({i: f"y{i+1}" for i in range(k)})
    amro_df               = pd.pivot(amro_df, index="date", columns="obs_name", values="num_positives").reset_index()
    for i in range(k):
        amro_df['oev'+str(i+1)] = 1 +(p_oev * amro_df['y'+str(i+1)].values)**2
    amro_df = amro_df.set_index("date")
    return amro_df

# infer using individual level observation, as indicated by the culture data.

In [ ]:
amro_search  = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE',  'PSEUDOMONAS AERUGINOSA',
                'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS', 'METHICILLIN-RESISTANT STAPHYLOCOCCUS AUREUS',
                "STAPHYLOCOCCUS EPIDERMIDIS", 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

path_to_amro = os.path.join(data_db_dir, "long_files_8_25_2021", "amro_ward.csv" )

for amro in amro_search:
    print("Running IF-EAKF for amro: ", amro2title(amro))
    path_to_save = os.path.join(results2_dir, "amro_inferences", "abm", f"{amro2cute(amro)}")
    os.makedirs(os.path.join(results2_dir, "amro_inferences", "abm"), exist_ok=True)

    gamma        = empirical_prevalence(amro, path_to_prev="../data/amro_prevalence.csv")

    if_settings["adjust_state_space"] = False

    path_to_samples = os.path.join(path_to_save, "infer_building", "individual_observation")
    os.makedirs(path_to_samples, exist_ok=True)

    if os.path.isfile(os.path.join(path_to_samples, f"{str(id_run).zfill(3)}posterior.npz")):
        continue

    alpha         = 1/120
    init_state    = lambda θ:       amr_abm(t = 0,
                                                    agents_state   = np.zeros((model_settings["n"], model_settings["m"])),
                                                    gamma          = gamma,
                                                    beta           = θ[1, :],
                                                    alpha          = alpha,
                                                    movement       = movement_df[movement_df["date"]==dates_simulation[0]],
                                                    ward2size      = ward2size,
                                                    model_settings = model_settings)

    process       = lambda t, x, θ: amr_abm(t = t,
                                                    agents_state   = x,
                                                    gamma          = gamma,
                                                    beta           = θ[1, :],
                                                    alpha          = alpha,
                                                    movement       = movement_df[movement_df["date"]==dates_simulation[t]],
                                                    ward2size      = ward2size,
                                                    model_settings = model_settings)

    obs_model = lambda t, x, θ: observe_cluster_individual(t = t,
                                                    agents_state   = x,
                                                    rho            = θ[0, :],
                                                    movement       = movement_df[movement_df["date"]==dates_simulation[t]],
                                                    model_settings = model_settings)

    obs_df = create_obs_building_amro(amro, model_settings, path_to_amro, resample="W-Sun")

    run_amro_inference(f               = process,
                        f0             = init_state,
                        g              = obs_model,
                        obs_df         = obs_df,
                        model_settings = model_settings,
                        if_settings    = if_settings,
                        id_run         = id_run,
                        path_to_save   = path_to_samples)


In [ ]:
import matplotlib.pyplot as plt


posterior_df = pd.DataFrame()
mle_df       = pd.DataFrame()

fig, axes = plt.subplots(len(amro_search), 2, figsize=(10, 12))
for idx_amro, amro in enumerate(amro_search):

    path_to_samples   = os.path.join(results2_dir, "amro_inferences", "abm", f"{amro2cute(amro)}", "infer_building")
    samples_inference = np.load(os.path.join(path_to_samples, f"{str(id_run).zfill(3)}posterior.npz"))

    θpost = samples_inference["posterior"]
    θmle  = samples_inference["mle"]
    Nif   = θpost.shape[-1]
    θpost = θpost[:, :, :, :].mean(-2)

    p_last_df           = pd.DataFrame(columns=["value", "param", "ens_id", "amro"])
    p_last_df["value"]  = np.concatenate([θpost[0, :, -1], θpost[1, :, -1]])
    p_last_df["param"]  = ["ρ"] * model_settings["m"] + ["β"] * model_settings["m"]
    p_last_df["ens_id"] = list(range(model_settings["m"])) * 2
    p_last_df["amro"]   = amro
    mle_df              = pd.concat([mle_df, p_last_df])

    ρ_post = θpost[0, :, :].flatten()
    β_post = θpost[1, :, :].flatten()

    post_df            = pd.DataFrame(columns=["value", "param", "ens_id", "if_iter", "amro"])
    post_df["value"]   = np.concatenate([ρ_post, β_post])
    post_df["param"]   = ["ρ"] * len(ρ_post) + ["β"] * len(β_post)
    post_df["if_iter"] = flatten_list([list(range(Nif)) * model_settings["m"] ] * 2 )
    post_df["ens_id"]  = flatten_list([[i] * Nif for i in range( model_settings["m"])] * 2)
    post_df["amro"]    = amro
    posterior_df       = pd.concat([posterior_df, post_df])

    ρ_df = create_df_response(θpost[0, :, :].T, time=Nif)
    β_df = create_df_response(θpost[1, :, :].T, time=Nif)

    p_dfs             = [ρ_df, β_df]
    param_label       = ["ρ", "β"]

    parameters_range  = np.array([[0.01, 20/100], [0, 0.15]])
    convergence_plot(θmle, p_dfs, parameters_range, param_label, ax=axes[idx_amro, :], fig=fig, param_truth=None)
    axes[idx_amro, 0].legend().remove(); axes[idx_amro, 1].legend().remove()
    axes[idx_amro, 1].set_xlabel(None)
    axes[idx_amro, 0].spines['right'].set_visible(False)
    axes[idx_amro, 0].spines['top'].set_visible(False)

    axes[idx_amro, 1].spines['right'].set_visible(False)
    axes[idx_amro, 1].spines['top'].set_visible(False)
    axes[idx_amro, 0].text(x=0.1, y=0.18, s=f"{amro2title(amro)}", weight='bold')

axes[0, 0].legend(loc="upper left", bbox_to_anchor=(0.0, 1.65), ncol=4)
axes[-1, 0].set_xlabel("Iteration")
axes[-1, 1].set_xlabel("Iteration")

fig.suptitle(f"Convergence plots\n Process model at ward level ($n=220$), inference at building level ($k=6$)\n, ABM", x=0.5)
plt.tight_layout()


In [1]:
import seaborn as sns

v_df              = posterior_df[posterior_df["if_iter"] == 20-1]
v_df["amro_plot"] = v_df["amro"].map( amro2title )
rho_df            = v_df[v_df["param"] == "ρ"]
beta_df           = v_df[v_df["param"] == "β"]
rho_df["value"]   = rho_df["value"] * 100

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
sns.violinplot(ax=ax[0], data=rho_df,  x="value", y="amro_plot", palette="Pastel1", inner=None, lw=0.5, color="k")
sns.violinplot(ax=ax[1], data=beta_df, x="value", y="amro_plot", palette="Pastel1", inner=None, lw=0.5, color="k")

amro2order =  {'E. coli': 0, 'K. pneumoniae': 1, 'P. aeruginosa': 2, 'MSSA': 3, 'MRSA': 4, 'S. epidermidis' : 5, 'E. faecalis' : 6, 'E. faecium' : 7}
rho_mean  = rho_df.groupby("amro_plot").mean().reset_index(); rho_mean["order"]   = rho_mean["amro_plot"].map( amro2order ); rho_mean = rho_mean.sort_values("order")
beta_mean = beta_df.groupby("amro_plot").mean().reset_index(); beta_mean["order"] = beta_mean["amro_plot"].map( amro2order ); beta_mean = beta_mean.sort_values("order")

sns.scatterplot(ax=ax[0], data=rho_mean,  x="value", y="amro_plot", fc="gray", ec=None, s=20)
sns.scatterplot(ax=ax[1], data=beta_mean,  x="value", y="amro_plot", fc="gray", ec=None, s=20)

ax[0].set_ylabel(None); ax[0].set_xlabel(r"$\rho$")
ax[1].set_ylabel(None); ax[1].set_xlabel(r"$\beta$")

for axi in ax:
    axi.spines['right'].set_visible(False)
    axi.spines['top'].set_visible(False)

fig.suptitle(f"ABM last iteration posterior", x=0.5)
plt.tight_layout()


/Users/chaosdonkey06/conda/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


NameError: name 'posterior_df' is not defined

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15.2, 7.2), sharey=False, sharex=False)
for idx_amro, ax in enumerate(axes.flatten()):
    p_df              = v_df[v_df.amro==amro_search[idx_amro]]
    rho_df            = p_df[p_df["param"] == "ρ"].reset_index(drop=True)
    beta_df           = p_df[p_df["param"] == "β"].reset_index(drop=True)
    rho_df["value"]   = rho_df["value"] * 100

    sns.kdeplot(ax    = ax,
                x     = rho_df["value"],
                y     = beta_df["value"],
                cmap  = "Reds",
                fill  = True,  thresh=0)

    ax.scatterplot(x  = rho_df["value"],
                   y  = beta_df["value"],
                   facecolor = "salmon",
                   edgecolor = "k", alpha=0.3)

    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_title(f"{amro2title(amro_search[idx_amro])}", weight='bold')

axes[0, 0].set_ylabel(r"$\beta$"); axes[1, 0].set_ylabel(r"$\beta$")
for i in range(4):
    axes[1, i].set_xlabel(r"$\rho$")

plt.tight_layout()

# infer using population level observations according to
$P_i(t)\sim \text{{Binom}}\left(T_i(t),\rho\cdot \frac{C(t)}{N(t)}\right)$

In [3]:
amro_search  = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE',  'PSEUDOMONAS AERUGINOSA',
                'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS', 'METHICILLIN-RESISTANT STAPHYLOCOCCUS AUREUS',
                "STAPHYLOCOCCUS EPIDERMIDIS", 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

path_to_amro = os.path.join(data_db_dir, "long_files_8_25_2021", "amro_ward.csv" )

for amro in amro_search:
    print("Running IF-EAKF for amro: ", amro2title(amro))
    path_to_save = os.path.join(results2_dir, "amro_inferences", "abm", f"{amro2cute(amro)}")
    os.makedirs(os.path.join(results2_dir, "amro_inferences", "abm"), exist_ok=True)

    gamma        = empirical_prevalence(amro, path_to_prev="../data/amro_prevalence.csv")

    if_settings["adjust_state_space"] = False

    path_to_samples = os.path.join(path_to_save, "infer_building", "population_observation")
    os.makedirs(path_to_samples, exist_ok=True)

    if os.path.isfile(os.path.join(path_to_samples, f"{str(id_run).zfill(3)}posterior.npz")):
        continue

    alpha         = 1/120
    init_state    = lambda θ:       amr_abm(t = 0,
                                                    agents_state   = np.zeros((model_settings["n"], model_settings["m"])),
                                                    gamma          = gamma,
                                                    beta           = θ[1, :],
                                                    alpha          = alpha,
                                                    movement       = movement_df[movement_df["date"]==dates_simulation[0]],
                                                    ward2size      = ward2size,
                                                    model_settings = model_settings)

    process       = lambda t, x, θ: amr_abm(t = t,
                                                    agents_state   = x,
                                                    gamma          = gamma,
                                                    beta           = θ[1, :],
                                                    alpha          = alpha,
                                                    movement       = movement_df[movement_df["date"]==dates_simulation[t]],
                                                    ward2size      = ward2size,
                                                    model_settings = model_settings)

    obs_model = lambda t, x, θ: observe_cluster_population(t = t,
                                                    agents_state   = x,
                                                    rho            = θ[0, :],
                                                    movement       = movement_df[movement_df["date"]==dates_simulation[t]],
                                                    model_settings = model_settings)

    obs_df = create_obs_building_amro(amro, model_settings, path_to_amro, resample="W-Sun")

    run_amro_inference(f               = process,
                        f0             = init_state,
                        g              = obs_model,
                        obs_df         = obs_df,
                        model_settings = model_settings,
                        if_settings    = if_settings,
                        id_run         = id_run,
                        path_to_save   = path_to_samples)


NameError: name 'data_db_dir' is not defined